In [1]:
# biblioteca usada para instalar e distribuir pacotes Python. # 
!pip install --upgrade setuptools 
!pip install --upgrade Pandas
!pip install --upgrade openpyxl 
!pip install --upgrade scikit-learn

In [2]:
import pandas as pd 

In [3]:
# caminho do arquivo #
path = r"C:\Users\nicol\OneDrive\Documents\Visual Studio - 2024\Estudos\Feature Engineering\ChavesClientesLimpo.xlsx"

# base com as informações #
base = pd.read_excel(path, engine="openpyxl") # escolhendo o motor que irá processar o arquivo #

# limpando o arquivo # 
base_clean = base.drop(['ChaveSituacao', 'ClassRisco', 'CatCliente'], axis = 1)

<h3> One Hot Encoding </h3>

In [4]:
# importando a biblioteca # 
from sklearn.preprocessing import OneHotEncoder

# atribuindo a uma variável #
ohe = OneHotEncoder() # (instanciando um objeto) #

# transformando as linhas em colunas # 
ohe_transformar = ohe.fit_transform(base_clean[['Genero','EstadoCivil']]) # precisamos passar um dataframe e não uma series #

# Nome das features #
print(ohe.get_feature_names_out())

['Genero_F' 'Genero_M' 'EstadoCivil_C' 'EstadoCivil_D' 'EstadoCivil_S'
 'EstadoCivil_V']


In [5]:
# Criando o dataframe com os valores criados # 
df_ohe = pd.DataFrame(ohe_transformar.toarray())

# Alterando o nome das colunas # 
df_ohe.columns = ohe.get_feature_names_out()

# concatenando o resultado ao dataframe inicial # 
base_clean = pd.concat([base_clean, df_ohe], axis = 1) # axis=1 para concatenar as colunas 

# Resultado: 
df_ohe.head(1)

,Genero_F,Genero_M,EstadoCivil_C,EstadoCivil_D,EstadoCivil_S,EstadoCivil_V
0,1.0,0.0,1.0,0.0,0.0,0.0


<h3> Ordinal Encoding </h3>

In [14]:
# importando a biblioteca # 
from sklearn.preprocessing import OrdinalEncoder

# atribuindo a uma variável #
oe = OrdinalEncoder(categories = [['Basic', 'Black', 'Platinum']]) # (instanciando um objeto) #

# transformando as linhas em colunas # 
oe_transformar = oe.fit_transform(base_clean.Categoria.values.reshape(-1, 1))
# .fit_transform(nome_df.coluna_ordenada.values.reshape(-1, 1)) 
# .values (queremos pegar apenas o valor e não o indice e o valor)
# .reshape(-1, 1) (precisamos ajustar já que os dados estao em um array e a função utiliza uma matriz bidimensional) #

# criar uma coluna com a informação gerada # 
base_clean['nCategoria'] = oe_transformar


In [18]:
# atribuindo a uma variável #
oe = OrdinalEncoder(categories = [['C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+']]) # (instanciando um objeto) #
# categories = [['Ordem', 'Dos', 'Valores']] 
# (definimos qual a ordem dos valores para na ordenação termos do menor para o maior) #

# transformando os valores e ordenando por ordem de grandeza # 
oe_transformar = oe.fit_transform(base_clean.Risco.values.reshape(-1, 1))
# .fit_transform(nome_df.coluna_ordenada.values.reshape(-1, 1)) 
# .values (queremos pegar apenas o valor e não o indice e o valor)
# .reshape(-1, 1) (precisamos ajustar já que os dados estao em um array e a função utiliza uma matriz bidimensional) #

# criar uma coluna com a informação gerada # 
base_clean['nRisco'] = oe_transformar


In [24]:
# Função para verificar clientes VIP's # 
def define_vip(valor): 
    if valor == 'Alpha':
        return 2 
    if valor == 'Beta': 
        return 1
    else: 
        return 0
    
# aplicando a função no dataframe # 
base_clean['nVIP'] = base_clean.CatVIP.apply(define_vip)


In [89]:
# importando a biblioteca # 
from sklearn.preprocessing import KBinsDiscretizer

# Instanciando o objeto KBinsDiscretizer # 
kbd = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
# n_bins = x  (Número de faixas (bins) desejadas para discretizar a variável contínua) 
# encode = 'x' Controla como os rótulos das faixas são codificados, os valores possíveis incluem 'onehot', 'onehot-dense', e 'ordinal'
# strategy = 'xxxx' (Estratégia usada para discretização)
# = 'uniform': Garante que os bins tenham a mesma largura
# = 'quantile': Garante que cada bin tenha aproximadamente a mesma quantidade de amostras
# = 'kmeans': Usa k-means clustering para definir as faixas #

# Criando as faixas # 
kbd_transformar = kbd.fit_transform(base_clean['Idade'].values.reshape(-1, 1))

# criar uma coluna com a informação gerada #
base_clean['nIdades'] = kbd_transformar


c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


In [92]:
base_final = base_clean.drop(['Idade', 'Genero', 'EstadoCivil', 'Categoria', 'CatVIP', 'Risco'], axis = 1)

In [93]:
base_final.head(10)

,Pagamento,Genero_F,Genero_M,EstadoCivil_C,EstadoCivil_D,EstadoCivil_S,EstadoCivil_V,nCategoria,nRisco,nVIP,nIdades
0,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2,3.0
1,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,7.0,0,0.0
2,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,1,1.0
3,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,0,0.0
4,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0.0
5,0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2,1.0
6,1,0.0,1.0,0.0,1.0,0.0,0.0,2.0,6.0,1,1.0
7,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0,3.0
8,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,6.0,0,1.0
9,1,0.0,1.0,0.0,0.0,0.0,1.0,2.0,2.0,0,3.0
